In [1]:
import logging
import time
from argparse import ArgumentParser
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as data
import torchvision.transforms as trans
import torch
from torch.utils.data import random_split
from os import listdir
from os.path import join, splitext, basename
import glob
from torch.utils.data import DataLoader
from PIL import Image
from os import listdir
from torch.utils.data import DataLoader
from torch.utils.data.sampler import Sampler
from PIL import Image
from imgaug import augmenters as iaa
from matplotlib.pyplot import figure, imshow, axis
import imgaug as ia
import numpy as np
import statistics
import random
import natsort
import copy
import collections
import torchvision.models as models
import torch.optim as optim
#import datasets
from macro import GeneralNetwork
from micro import MicroNetwork
from nni.nas.pytorch import enas
from nni.nas.pytorch.callbacks import (ArchitectureCheckpoint,
                                       LRSchedulerCallback)
from utils import accuracy, reward_accuracy
from util.cutout import Cutout

In [2]:
#To determine if your system supports CUDA
print("==> Check devices..")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Current device: ",device)

#Also can print your current GPU id, and the number of GPUs you can use.
print("Our selected device: ", torch.cuda.current_device())
print(torch.cuda.device_count(), " GPUs is available")

==> Check devices..
Current device:  cuda
Our selected device:  0
1  GPUs is available


In [3]:
class Config():
    def __init__(self):
        self.FolderNames2English_names = {
                                    "Pepper__bell___Bacterial_spot":0, 
                                    "Pepper__bell___healthy":1,
                                    "Potato___Early_blight":2, 
                                    "Potato___healthy":3,
                                    "Potato___Late_blight":4, 
                                    "Tomato_Bacterial_spot":5,                               
                                    "Tomato_Early_blight":6,        
                                    "Tomato_healthy":7,           
                                    "Tomato_Late_blight":8,        
                                    "Tomato_Leaf_Mold":9,          
                                    "Tomato_Septoria_leaf_spot":10,
                                    "Tomato_Spider_mites_Two_spotted_spider_mite":11,
                                    "Tomato__Target_Spot":12,
                                    "Tomato__Tomato_mosaic_virus":13,
                                    "Tomato__Tomato_YellowLeaf__Curl_Virus":14,
                                    }
        self.folder_names2code = {}
        self.early_stop = 5
        self.max_epoch = 1000
        self.train_batchsize = 256
        self.eva_val_batchsize = 64
        self.class_num = 15
        self.each_class_item_num = {}
        self.temperature = 1
        self.alpha = 0.5
        
        self.image_size = 256  # resolution 

        
        self.train_dataset_path = r'/root/wei/DL_sys/Final_project/train'
        self.test_dataset_path = r'/root/wei/DL_sys/Final_project/validation'
        self.eval_dataset_path = r'/root/wei/DL_sys/Final_project/test'
        self.model_ouput_dir = './model/'
        
        self.save_name_rec = ''  #layer_resolution_groups_groupwidth
        self.best_epoch = 0
        
        self.net = 'MyResNeXt'  # 0: resnet18/MyResNeXt
        self.pretrain = False

        self.lr = 0.00001
        self.criterion = nn.CrossEntropyLoss()

In [4]:
class ImgAugTransform():
    def __init__(self, config=Config()):
        self.aug = iaa.Sequential([
            iaa.Scale((config.image_size, config.image_size)),
            iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),
            iaa.Fliplr(0.5),
            iaa.Affine(rotate=(-20, 20), mode='symmetric'),
            iaa.Sometimes(0.25,
                      iaa.OneOf([iaa.Dropout(p=(0, 0.1)),
                                 iaa.CoarseDropout(0.1, size_percent=0.5)])),  iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)  
        ])
      
    def __call__(self, img):
        img = np.array(img)
        return self.aug.augment_image(img)   

In [5]:
class PlantDisease_Dataset(data.Dataset):
    def __init__(self, image_dir, 
                 input_transform, is_train=False, config=Config()):
        super(PlantDisease_Dataset, self).__init__()
        
        class_list = glob.glob(image_dir + '/*', recursive=True)
        
        self.datapath = image_dir
        self.image_filenames = []
        self.num_per_classes = {}
        self.class_dict_list = {}
        #print(len(files_list))
        
        for class_name in class_list:
            file_list = glob.glob(class_name + '/*', recursive=True)
            #print(class_name)
            
            for file in file_list:
                #print(file)
                self.image_filenames.append(file)
                class_name = class_name.split("/")[-1]
                if is_image_file(file):
                    if class_name in self.num_per_classes:
                        self.num_per_classes[class_name] += 1
                        self.class_dict_list[class_name].append(file)
                    else:
                        self.num_per_classes[class_name] = 1
                        self.class_dict_list[class_name] = []
                        self.class_dict_list[class_name].append(file)
        #print(self.num_per_classes)

        self.input_transform = input_transform
        
    def __getitem__(self, index):
        input_file = self.image_filenames[index]
        img = Image.open(input_file)
        img = img.convert('RGB')
        if self.input_transform is not None:
            img = self.input_transform(img)
            
        class_name = input_file.split('/')[-2]
        label = config.FolderNames2English_names[str(class_name)]
        return img, label

    def __len__(self):
        return len(self.image_filenames)
    
    def show_details(self):
        for key in sorted(self.num_per_classes.keys()):
            print("{:<50}|{:<20}|{:<5}".format(
                key,
                config.FolderNames2English_names[key],
                self.num_per_classes[key]
            ))
            
    def compute_weights(self):
        od_train_dataset_num_per_classes = self.num_per_classes.items()
        #print(list(od_train_dataset_num_per_classes)[0][1])
   
        wts = []
        prob = {}
        
        for i in range(len(list(od_train_dataset_num_per_classes))):
            if list(od_train_dataset_num_per_classes)[i][0] not in prob:
                prob[list(od_train_dataset_num_per_classes)[i][0]] = 1/list(od_train_dataset_num_per_classes)[i][1]
        
        for input_file in self.image_filenames :
            class_name = input_file.split('/')[-2]
            wts = np.append(wts,prob[class_name])
        
        print("Weighted Sample Ratio",wts,len(wts))
        return wts

In [6]:
def evaluation(model,eval_dataset , evaluation_loader, config):
    test_loss = 0.0
    correct_eval = 0
    eval_num = 0
    cls = np.zeros(config.class_num)
    correct_top3 = 0

    model.eval()
    num_per_classes = {}
    ori_num_per_classes = {}
    err = {}
     

    for i, (inputs, labels) in enumerate(evaluation_loader, 0):
        # move tensors to GPU if CUDA is available
        inputs = inputs.to(device) 
        labels = labels.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(inputs)
        pred = outputs.max(1)
              
        
        for i in range(len(labels)):
            if pred[1][i] == labels[i]:
                correct_eval += 1
            else:
                if labels[i].item() in err:
                    err[ labels[i].item()] +=1
                else:
                    err[ labels[i].item()]  = 1
            
            labels[i].item()
            if labels[i].item() in ori_num_per_classes:
                ori_num_per_classes[ labels[i].item()] +=1
            else:
                ori_num_per_classes[ labels[i].item()]  = 1
            
            #print(pred[1][i].item())
            if pred[1][i].item() in num_per_classes:
                num_per_classes[pred[1][i].item()] += 1
            else:
                num_per_classes[pred[1][i].item()] = 1

            eval_num +=1
            
        
    #print(correct_eval)
    #print(eval_num)
    #print((num_per_classes))
    #print((ori_num_per_classes))
    #print((err))
    '''
    for i in err:
        for j in ori_num_per_classes:
            if i == j :
                print (i, " ",(ori_num_per_classes[i]-err[i])/ori_num_per_classes[i])
    '''
    print('Evaluation accuracy: %.4f' % (correct_eval / eval_num))
    print('-----------------------------------------')

In [7]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".JPG",".png", ".jpg", ".jpeg"])

In [8]:
random.seed(30)
if __name__ == '__main__':
    config = Config()
    
    transform_test = trans.Compose([
        trans.Resize((config.image_size, config.image_size)),
        trans.ToTensor(),
        trans.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    
    eval_dataset = PlantDisease_Dataset(image_dir = config.eval_dataset_path ,input_transform=transform_test, config=config)
    
    evaluation_loader = DataLoader(eval_dataset, batch_size=8, num_workers=1)
    
    #load_name = "model/152.pth"
    #net = torch.load(load_name)
    #print(net) 
    #evaluation(net,eval_dataset , evaluation_loader, config)
    
    
    
    for i in range(199):
        print("This is model: "+str(i))
        load_name = "model/" + str(i) + ".pth"
        net = torch.load(load_name)
        #print(net) 
        evaluation(net, eval_dataset, evaluation_loader, config)
    

This is model: 0
Evaluation accuracy: 0.0681
-----------------------------------------
This is model: 1
Evaluation accuracy: 0.1221
-----------------------------------------
This is model: 2
Evaluation accuracy: 0.3315
-----------------------------------------
This is model: 3
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 4
Evaluation accuracy: 0.0775
-----------------------------------------
This is model: 5
Evaluation accuracy: 0.1119
-----------------------------------------
This is model: 6
Evaluation accuracy: 0.1003
-----------------------------------------
This is model: 7
Evaluation accuracy: 0.1536
-----------------------------------------
This is model: 8
Evaluation accuracy: 0.1250
-----------------------------------------
This is model: 9
Evaluation accuracy: 0.1323
-----------------------------------------
This is model: 10
Evaluation accuracy: 0.1296
-----------------------------------------
This is model: 11
Evaluation accuracy: 0.1

Evaluation accuracy: 0.2343
-----------------------------------------
This is model: 95
Evaluation accuracy: 0.6978
-----------------------------------------
This is model: 96
Evaluation accuracy: 0.4509
-----------------------------------------
This is model: 97
Evaluation accuracy: 0.0778
-----------------------------------------
This is model: 98
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 99
Evaluation accuracy: 0.1049
-----------------------------------------
This is model: 100
Evaluation accuracy: 0.6535
-----------------------------------------
This is model: 101
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 102
Evaluation accuracy: 0.4706
-----------------------------------------
This is model: 103
Evaluation accuracy: 0.2152
-----------------------------------------
This is model: 104
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 105
Evaluation accuracy: 0.077

Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 188
Evaluation accuracy: 0.5018
-----------------------------------------
This is model: 189
Evaluation accuracy: 0.0647
-----------------------------------------
This is model: 190
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 191
Evaluation accuracy: 0.3339
-----------------------------------------
This is model: 192
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 193
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 194
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 195
Evaluation accuracy: 0.4904
-----------------------------------------
This is model: 196
Evaluation accuracy: 0.6787
-----------------------------------------
This is model: 197
Evaluation accuracy: 0.0771
-----------------------------------------
This is model: 198
Evaluation accuracy: 

model 152 0.8459